In [ ]:
import matplotlib.pyplot as plt
import pyproj
import meshio
import folium
import femlium

Auxiliary function to get a `folium` `Map` close to Lake Garda.

In [ ]:
def get_garda_geo_map(boundary_icons=False):
    # Add map close to Lake Garda
    geo_map = folium.Map(location=[45.6389113, 10.7521368], zoom_start=10.3)

    # Add markers
    if boundary_icons:
        location_markers = {
            "Sarca": [45.87395405, 10.87087005],
            "Mincio": [45.43259035, 10.7007715]
        }
        location_colors = {
            "Sarca": "red",
            "Mincio": "green"
        }

        for key in location_markers.keys():
            folium.Marker(
                location=location_markers[key],
                tooltip=key,
                icon=folium.Icon(color=location_colors[key])
            ).add_to(geo_map)

    # Return folium map
    return geo_map

In [ ]:
get_garda_geo_map()

Read the mesh from file with `meshio`.

In [ ]:
mesh = meshio.read("data/garda.msh")

Plot the mesh using `matplotlib`.

In [ ]:
fig = plt.figure(figsize=(12, 12))
fig.gca().triplot(mesh.points[:, 0], mesh.points[:, 1], mesh.cells_dict["triangle"])
fig.gca().axis("equal")

Define a `pyproj` `Transformer` to map between different reference systems, because the points read from file are stored a $(x, y)$ pairs in the EPSG32632 reference system, while the map produced by `folium` is based on (latitude, longitude) pairs in the EPSG4326 reference system.

In [ ]:
transformer = pyproj.Transformer.from_crs("epsg:32632", "epsg:4326", always_xy=True)

We define a mesh plotter for meshes in `meshio` format, which is implemented in `femlium.MeshioPlotter`.

In [ ]:
mesh_plotter = femlium.MeshioPlotter(transformer)

We use the `mesh_plotter` to draw the mesh on top of the geographic map.

In [ ]:
geo_map = get_garda_geo_map()
mesh_plotter.add_mesh_to(geo_map, mesh)
geo_map

We may change the color and the weight of the line.

In [ ]:
geo_map = get_garda_geo_map()
mesh_plotter.add_mesh_to(geo_map, mesh, face_colors="red", face_weights=2)
geo_map

Furthermore, we may set the colors and the weights of the face representation to depend on the markers associated to each segment.

In [ ]:
geo_map = get_garda_geo_map(boundary_icons=True)
face_colors = {
    0: "gray",
    1: "blue",
    2: "red",
    3: "green"
}
face_weights = {
    0: 1,
    1: 2,
    2: 5,
    3: 5
}
mesh_plotter.add_mesh_to(geo_map, mesh, face_colors=face_colors, face_weights=face_weights)
geo_map

Cells can be colored as well, with a uniform color or depending on the cell markers. We start from a uniform color.

In [ ]:
geo_map = get_garda_geo_map()
mesh_plotter.add_mesh_to(geo_map, mesh, cell_colors="orange")
geo_map

We also show the case of colors being set from cell markers. Theare two cell markers in this mesh, equal to 1 for the region close to the shoreline (colored in purple) and 2 for the rest of the domain (colored in yellow).

In [ ]:
geo_map = get_garda_geo_map()
cell_colors = {
    1: "purple",
    2: "yellow"
}
mesh_plotter.add_mesh_to(geo_map, mesh, cell_colors=cell_colors)
geo_map

Once can use colors associated to both cell and face markers on the same plot. 

In [ ]:
geo_map = get_garda_geo_map(boundary_icons=True)
mesh_plotter.add_mesh_to(
    geo_map, mesh, cell_colors=cell_colors, face_colors=face_colors, face_weights=face_weights)
geo_map